# Notebook Lecture 7: Feedback and the Root Locus Method
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook introduces the concept of feedback control and presents a first method for feedback control analysis with the Root Locus method.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives
Previously we have discussed how to express the input-output behaviour of a system. However, as seen in Lecture 1, the system behaviour may be undesired (i.e., unstable). Consequentially, in this lecture we introduce feedback control, where the aim is to be able to analyze a system behaviour, and then correspondingly design controllers such that the system behaviour achieved some specified objective.

After completing this material, you should be able to:

- Understand how feedback control modifies system behavior:
 	- Stabilizing unstable systems
 	- Improving responsiveness
 	- Reducing oscillations

- Apply the Root Locus method to:
 	- Determine closed-loop pole locations for varying gains
 	- Analyze system stability and performance


### Import the packages:

The following cell imports the required packages. Run it before running the rest of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from control import tf, rlocus, poles, feedback, impulse_response
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact
import warnings

## Open Loop vs Closed Loop Transfer Functions

As we have seen in the last lecture, a transfer function maps the input to the output of our system in the frequency domain.

Let's look at an example:

## Feedback Control System Behavior

Feedback control is a powerful technique that can fundamentally alter how a system behaves. By measuring the system's output and comparing it to a desired reference, a controller can adjust the system's input to achieve the desired output. This simple concept has far-reaching implications for system behavior, allowing us to stabilize unstable systems, improve responsiveness, and reduce oscillations.

### 1. Stabilizing Unstable Systems 

Recall the inverted pendulum with:
- Length: L = $\frac{3}{2}$
- Mass: m = $1$
- Moment of inertia: J = $\frac{1}{3}mL^2$ = $\frac{3}{4}$
- Damping coefficient: c = $\frac{9}{4}$
- Input torque: $\tau_f$

<div style="text-align:center;">
    <img src="./img/pendulum.png" alt="Inverted Pendulum" width="500">
</div>

The transfer function when linearized around the unstable equilibrium point is:

$$G(s) = \frac{3}{s^2 + 3s - 10}$$

#### Open Loop Characteristics

<div style="text-align:center;">
    <img src="./img/OL.png" alt="Inverted Pendulum" width="500">
</div>

The characteristic equation is:
$$s^2 + 3s - 10 = 0$$

The poles of the system are:
$$\lambda_1 = 2, \quad \lambda_2 = -5$$

As we see we have an unstable system, which makes sense are we are linearizing around the unstable upright position and therefore any small perturbation will make the system unstable.

#### Closed Loop Characteristics

Now let's add a feedback controller to stabilize the system. We will use a proportional controller with gain $K$:

$$G_c(s) = K$$

<div style="text-align:center;">
    <img src="./img/CL.png" alt="Inverted Pendulum" width="500">
</div>

The closed-loop transfer function is:

$$G_{cl}(s) = \frac{G(s)}{1 + G(s)G_c(s)} = \frac{3K}{s^2 + 3s - 10 + 3K}$$


The characteristic equation is:

$$s^2 + 3s - 10 + 3K = 0$$

The poles of the system are:

$$\lambda_{1,2} = \frac{-3 \pm \sqrt{3^2 + 4(10 - 3K)}}{2}$$

To achieve asymptotic stability, we need all eigenvalues to have strictly negative real part. Therefore, the system is stable if:

$$49 - 12K < 9 \Rightarrow K > \frac{10}{3}$$


In [ ]:
def plot_system_analysis(K, mode):
   # Create figure with two subplots side by side, fixed square size
   fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
   
   # System definition
   num = [3]
   den = [1, 3, -10]
   sys = tf(num, den)
   
   # Plot 1: Impulse Response
   if mode == 'Open-Loop':
       t, y = impulse_response(sys)
       title1 = 'Open-Loop Impulse Response'
       label = 'Open-Loop'
       color = 'green'
       current_sys = sys
   else:
       closed_loop_sys = feedback(K * sys, 1)
       t, y = impulse_response(closed_loop_sys)
       title1 = f'Closed-Loop Impulse Response (K={K})'
       label = f'Closed-Loop (K={K})'
       color = 'blue'
       current_sys = closed_loop_sys
   
   ax1.plot(t, y, label=label, color=color, lw=2)
   ax1.set_title(title1)
   ax1.set_xlabel('Time (s)')
   ax1.set_ylabel('Response')
   ax1.grid(True)
   ax1.legend()
   
   # Plot 2: Pole Locations
   poles = current_sys.poles()
   
   # Create pole plot
   ax2.scatter(poles.real, poles.imag, marker='x', s=100, color=color)
   
   # Make the pole plot square with fixed limits
   limit = 10
   ax2.set_xlim(-limit, limit)
   ax2.set_ylim(-limit, limit)
   
   # Add lines for real and imaginary axes
   ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
   ax2.axvline(x=0, color='k', linestyle='-', alpha=0.3)
   
   # Set equal aspect ratio and grid
   ax2.grid(True)
   ax2.set_aspect('equal', adjustable='box')
   
   # Add title and labels
   if mode == 'Open-Loop':
       title2 = 'Open-Loop Pole Locations'
   else:
       title2 = f'Closed-Loop Pole Locations (K={K})'
   
   ax2.set_title(title2)
   ax2.set_xlabel('Real Part')
   ax2.set_ylabel('Imaginary Part')
   
   # Add labels for poles
   for pole in poles:
       ax2.annotate(f'({pole.real:.2f}, {pole.imag:.2f}j)',
                   (pole.real, pole.imag),
                   xytext=(10, 10), textcoords='offset points')
   
   plt.tight_layout()

def interact_with_system_analysis():
    K_slider = widgets.FloatSlider(
        min=0, max=35, step=0.1, value=1,
        description='Gain (K):'
    )
    
    mode_button = widgets.ToggleButtons(
        options=['Open-Loop', 'Closed-Loop'],
        description='System Mode:',
        disabled=False
    )
    
    # Create an Output widget to hold the K slider
    K_container = widgets.Output()
    
    def on_mode_change(change):
        K_container.clear_output()
        if change['new'] == 'Closed-Loop':
            with K_container:
                display(K_slider)
    
    mode_button.observe(on_mode_change, names='value')
    
    ui = widgets.VBox([mode_button, K_container])
    
    out = widgets.interactive_output(plot_system_analysis, {
        'K': K_slider,
        'mode': mode_button
    })
    
    display(ui, out)

# Call the interactive function
interact_with_system_analysis()

It can be clearly seen in the closed loop answer, that when K is greater than 10/3 the system is stable.

A different approach to analyze the stability of the system is through looking at the closed loop poles of the system with varying K. This is done through the Root Locus method.

## Dynamic Compensators

To control feedback systems, we as control system designers have one main job: designing the dynamic compensator. This is the controller that we can put between the input and the object we actually want to control, often called the plant. While we will mainly treat these dynamic compensators as transfer functions, they often represent custom analog electronics (in the past) or programs running on a microcontroller.

In the example above we used the simple proportional controller:
$$
C(s) = K
$$
This is of course a very simple dynamic compensator and far from the only one we can use. Two more sophisticated dynamic compensators that we will treat in detail in this course are:
- PID control: 	$$C(s) = K_P + \frac{K_I}{s} + K_Ds$$
- Lead-lag: 	$$C(s) = K \frac{s+z}{s+p}$$

With $G(s) = \frac{1}{s}$ and $G(s) = s$ being the transfer functions of the integrator and differentiator respectively.

To design a dynamic compensator, we have different tools at our disposal. In the following, we will look at one of them, namely the Root Locus Method.

## Root Locus Method

The Root Locus method is a graphical technique in control theory to analyze and design control systems. 
It helps in determining how the poles of the closed-loop system vary as the gain (or another system parameter) is varied. This is important to determine if the system is stable or meets our design criteria.

We will now plot the Root Locus in two different ways.

In the first plot, you can enter an open loop transfer function and then change the gain k and see how that changes the poles and zeros.

In [ ]:
# Global variable to store the last transfer function
previous_num = ''
previous_den = ''
pole_trajectories = []

# Function to compute poles for a given K and open-loop transfer function
def plot_poles_with_K(K, num_str, den_str):
    global previous_num, previous_den, pole_trajectories

    # Check if the transfer function has changed
    if num_str != previous_num or den_str != previous_den:
        pole_trajectories = []  # Reset the pole trajectory
        previous_num = num_str  # Update the stored numerator
        previous_den = den_str  # Update the stored denominator

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Closed-loop transfer function for negative feedback
    closed_loop_sys = feedback(K * sys, 1)
    
    # Get poles of the closed-loop system
    poles = closed_loop_sys.poles()
    
    # Append the current poles to the trajectory list
    pole_trajectories.append(poles)
    
    # Get the real and imaginary parts of all the poles in the trajectory
    all_real_parts = [np.real(p) for traj in pole_trajectories for p in traj]
    all_imag_parts = [np.imag(p) for traj in pole_trajectories for p in traj]
    
    # Determine the dynamic plot limits with a margin
    real_min, real_max = min(all_real_parts) - 1, max(all_real_parts) + 1
    imag_min, imag_max = min(all_imag_parts) - 1, max(all_imag_parts) + 1

    # Plot the poles
    plt.figure()
    plt.axhline(0, color='black', lw=1)
    plt.axvline(0, color='black', lw=1)
    
    # Plot all previous poles to form the "trajectory"
    for previous_poles in pole_trajectories:
        plt.scatter(np.real(previous_poles), np.imag(previous_poles), color='blue', alpha=0.5)
    
    # Plot the current poles in red
    plt.scatter(np.real(poles), np.imag(poles), color='red', zorder=5, label=f'K = {K}')
    
    plt.grid(True)
    plt.title(f'Poles for K = {K}')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.xlim([real_min, real_max])
    plt.ylim([imag_min, imag_max])
    plt.legend()
    plt.show()

# Interactive function for students to input open-loop transfer function and vary K
def interact_with_poles():
    global pole_trajectories
    pole_trajectories = []  # Initialize the trajectory
    
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Slider to vary K
    K_slider = widgets.FloatSlider(min=0, max=1000, step=0.1, value=1, description='Gain (K):')
    
    # Combine input boxes and slider in a vertical layout
    ui = widgets.VBox([num_box, den_box, K_slider])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_poles_with_K, {
        'K': K_slider,
        'num_str': num_box,
        'den_str': den_box
    })
    
    # Display the UI and output
    display(ui, out)

# Call the interactive function
interact_with_poles()


If you do this for every k, you will have drawn the root locus. To show that, you can now enter the same transfer function and use the Root Locus function built-in in the Controls library. You will see that every point you plotted above lies on the Root Locus.

In [ ]:
# Function to plot the root locus for the given transfer function
def plot_root_locus(num_str, den_str):
    # Suppress warnings for this function
    warnings.filterwarnings("ignore")

    # Convert string input to lists of floats
    num = [float(n) for n in num_str.split()]
    den = [float(d) for d in den_str.split()]
    
    # Create the open-loop transfer function
    sys = tf(num, den)
    
    # Plot the root locus
    plt.figure()
    rlocus(sys, plot=True)
    plt.title('Root Locus Plot')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(False)
    plt.xlim([-5, 5])
    plt.ylim([-5, 5])
    plt.show()

# Interactive function for students to input transfer function and plot the root locus
def interact_with_root_locus():
    # Input boxes for open-loop transfer function numerator and denominator
    num_box = widgets.Text(value='1', description='Numerator:', placeholder='Enter numerator coefficients (space-separated)')
    den_box = widgets.Text(value='1 2 1', description='Denominator:', placeholder='Enter denominator coefficients (space-separated)')
    
    # Combine input boxes in a vertical layout
    ui = widgets.VBox([num_box, den_box])
    
    # Connect the input and output with the plot function
    out = widgets.interactive_output(plot_root_locus, {
        'num_str': num_box,
        'den_str': den_box
    })
    
    # Display the UI and output
    display(ui, out)

# Call the function to plot root locus
interact_with_root_locus()

As you see, it is quite easy to draw the root locus on a computer. However, to gain intuition it is important to be able to draw the Root Locus by hand. To do that, there is a certain set of rules:

1. The closed-loop poles need to be symmetric with respect to the real axis (i.e., either real, or complex-conjugate pairs).

2. The number of closed-loop poles is the same as the number of open-loop poles.

3. At $k\rightarrow 0$, the closed-loop poles approach the open-loop poles.

4. At $k\rightarrow \infty$, the closed-loop poles approach the open-loop zeros. Excess closed-loop poles go to infinity.

5. All points on the real axis are on the Root Locus:
	- all points on the real axis to the left of an odd number of poles/zeros are on the positive k Root Locus (the one we usually consider)
	- all points on the real axis to the left of an even number of poles/zeros are on the negative k Root Locus
	
6. If the number of poles is greater than the number of zeros, there are asymptotes:
	- asymptotes intersect the real axis at: $$\sigma = \frac{\Sigma p_i - \Sigma z_i}{\# poles - \# zeros}$$
	- asymptotes radiate out with angle: 	$$\angle s = \frac{180^\circ \pm q \cdot 360^\circ}{\# poles - \# zeros} \text{ if } k>0$$
											$$\angle s = \frac{\pm q \cdot 360^\circ}{\# poles - \# zeros} \text{ if } k<0$$
											with $q = 1, \ldots, (\# poles - \# zeros)$


To get a sense of where these rules come from, you can refer to the lecture or [this video series](https://youtube.com/playlist?list=PLUMWjy5jgHK3-ca6GP6PL0AgcNGHqn33f&si=ba0iUjapBkxisq9B).